In [1]:
import datetime
import itertools
import tqdm
import pandas
import pandas_market_calendars as mcal

In [4]:
cd ..

C:\Users\chlan\Projects\Prometheus


In [5]:
from prometheus.datasource import api
from prometheus import utils

In [6]:
dat = pandas.read_csv('Z:/Projects/knowsec/data/securities_prices_v2.tsv', delimiter='\t')

In [7]:
dat_sub = dat[dat.ticker.isin(['AAPL', 'AMZN', 'TSM', 'AMD'])]
dat_tsm = dat[dat.ticker == 'AAPL']

In [8]:
DSOURCE = api.SecuritySource()

In [9]:
df = dat_tsm

nyse_cal = nyse = mcal.get_calendar('NYSE')
nasdaq_cal = nyse = mcal.get_calendar('NASDAQ')

In [10]:
def query_missing_stocks(df):
    min_date = df.date.min()
    upto_date = utils.general.yesterday(tz='EST').date()
    
    # These are the dates we expect to see in the dataframe
    expected_dates = nyse_cal.schedule(start_date=min_date, end_date=upto_date)
    expected_dates = [i.date() for i in expected_dates.index.tolist()]
    
    # dates that are currently stored in the dataframe
    stored_dates = [datetime.datetime.fromisoformat(i).date() for i in df.date.tolist()]
    
    # these are the dates we need to find because they're missing
    need_to_query = [i for i in expected_dates if i not in stored_dates]
    need_to_query.sort()
    
    if need_to_query:
        ticker = df.ticker.unique().tolist()[0]

        gb = itertools.groupby(need_to_query, key=lambda x: x.year)

        result = list()

        for year, the_dates in gb:
            the_dates = list(the_dates)

            if len(the_dates) == 1:
                query_min_date = the_dates[0]
                query_max_date = the_dates[0] + datetime.timedelta(days=1)

            else:
                query_min_date = the_dates[0]
                query_max_date = the_dates[-1] + datetime.timedelta(days=1)
                
            try:
                stock_dat = DSOURCE.get_stock_historical(
                    symbol=ticker, start=query_min_date, end=query_max_date,
                    debug=False
                )
            except AttributeError:
                # Error thrown for when date range has no data
                stock_dat = pandas.DataFrame(
                    columns=['ticker', 'open', 'high', 'low', 'close', 'volume',
                             'dividends', 'stock_splits'],
                    index=pandas.DatetimeIndex(data=the_dates))
                
            stock_dat.index.set_names('date', inplace=True)
            stock_dat = stock_dat.drop('ticker', axis=1)

            result.append(stock_dat)
            
        semi_result = pandas.concat(result).sort_index()
        
        row_keeps = [True if i in need_to_query else False for i in semi_result.index]
        semi_result = semi_result.loc[row_keeps,:]
            
    return semi_result

In [100]:
from tqdm.notebook import tqdm_notebook

In [101]:
n_tickers = dat.ticker.unique().size
pb = tqdm_notebook(dat.groupby('ticker'), total=n_tickers)

  0%|          | 0/18075 [00:00<?, ?it/s]

In [102]:
for ticker, stock_series in pb:
    result = query_missing_stocks(stock_series_output)
    result = result.dropna(how='all')
    
    if result.empty is not True:

        date_yesterday = utils.general.yesterday(tz='EST').date()
        date_yesterday = date_yesterday.isoformat()
        filename = f"data/stock fillings/{ticker}_v{date_yesterday}.tsv"

        result.to_csv(filename, sep='\t')

    else:

        pass

ConnectionError: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/ACT?period1=1420174800&period2=1451624400&interval=1d&includePrePost=False&events=div%2Csplits (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028AA6AFB5B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))